<a href="https://colab.research.google.com/github/chandanareddy-enugala/NLP-SLU/blob/main/nlp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

import os
from keras.models import Model
from keras.layers import Input, LSTM, Dense, GRU
import matplotlib.pyplot as plt
import seaborn as sns

from keras.utils import to_categorical
from numpy import argmax


import re
import os
import sys
import numpy as np
import time
import math
from datetime import datetime

### **Defining Dictionary Functions Class**

### **Loading Dataset**

In [ ]:
def load_data(filePath):
  data_cwe = open(filePath, 'r').read().lower()
  return data_cwe

### **Preprocessing Model Preparation**

In [ ]:
class Language_Translation_Model:
  def __init__(self, lang="") -> None:
    self.lang = lang
    self.data = pd.DataFrame(columns=['sentence'])
    self.all_Unique_Characters = None
    self.length_all_Unique_Characters = None
    self.length_max_sentence = None
    self.sentenceArray_to_OHE = None
    self.char_to_num = None
    self.num_to_char = None
    # self.ASCII_Chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
  
  def sentence_to_array(self, sentence):
    sentenceArray = [self.char_to_num[char] for char in sentence]
    #padding = [0 for _ in range(self.length_all_Unique_Characters-len(sentenceArray))]
    #sentenceArray = np.array(sentenceArray+padding)
    return sentenceArray
  
  def sentenceArray_to_OHE1(self, sentenceArray):
    sentenceArray_OHE = to_categorical(sentenceArray)
    (req_rows, req_cols) = (self.length_max_sentence, self.length_all_Unique_Characters)
    (actual_rows, actual_cols) = sentenceArray_OHE.shape
    sentenceArray_OHE = np.concatenate((sentenceArray_OHE, np.zeros((req_rows-actual_rows, actual_cols), dtype='float32')), axis=0)
    sentenceArray_OHE = np.concatenate((sentenceArray_OHE, np.zeros((req_rows, req_cols-actual_cols), dtype='float32')), axis=1)
    return sentenceArray_OHE

  def OHE_to_sentenceArray(self, sentence_array_OHE):
    sentenceArray = []
    for OHE in sentence_array_OHE:
      if sum(OHE)!=0:
        sentenceArray.append(argmax(OHE))
    return sentenceArray
  
  def preprocess_data(self, text):
    # Cleaning --------------------------
    text = text.replace("\n", " ") 
    for char in [' .', ' ,', ' ;', ' ?', ' !', '( ', ' )']:
      text = text.replace(char, char.strip())
    text = text.replace("<s>", "")
    
    # =========================== CHARACTERS ===========================
    # Unique Characters -----------------
    self.all_Unique_Characters = list(set(text.replace("</s>", "")))
    self.all_Unique_Characters.sort()
    
    # Max Length of Unique Characters ---
    self.length_all_Unique_Characters = len(self.all_Unique_Characters)
    
    # Preparing Character to Number and vice-versa -----
    self.char_to_num = {char:num for num, char in enumerate(self.all_Unique_Characters)}
    self.num_to_char = {val:key for key, val in self.char_to_num.items()}

    # =========================== SENTENCES ===========================
    # Sentences -------------------------
    sentences = text.split("</s>")[:-1]
    sentences = [sentence.strip() for sentence in sentences]
    
    # Max Length of Sentences -----------
    self.length_max_sentence = max([len(sentence) for sentence in sentences])
    
    # Preparing OHE Array Variable ------
    self.sentenceArray_to_OHE = np.zeros((self.length_max_sentence, self.length_all_Unique_Characters), dtype='float32')
    
    # =========================== DATAFRAME ===========================
    # Storing sentences in dataframe ----
    self.data['sentence'] = sentences
    # Converting Sentence to Array Numbers ----
    self.data['sentence_array'] = self.data['sentence'].apply(lambda x: self.sentence_to_array(x))
    # Converting Array Numbers to One-Hot-Encoding (OHE) ----
    self.data['sentence_array_OHE'] = self.data['sentence_array'].apply(lambda x: self.sentenceArray_to_OHE1(x))

    return self.data

In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-source.txt"
train_input_data = load_data(filePath)

In [ ]:
Input_LTM = Language_Translation_Model()
Input_data = Input_LTM.preprocess_data(train_input_data[0:10000])
Input_data.head()

,sentence,sentence_array,sentence_array_OHE
0,"cinnte go leór, thiocfadh dóbhtha bás a fhaghá...","[16, 22, 26, 26, 31, 18, 0, 20, 27, 0, 24, 18,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,(bhí sé follasach go rabh an poll sin ag foscl...,"[4, 15, 21, 36, 0, 30, 35, 0, 19, 27, 24, 24, ...","[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
2,) d'fhéadfadh siad bás ' fhagháil ar a bhruach...,"[5, 0, 17, 3, 19, 21, 35, 14, 17, 19, 14, 17, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,..."
3,thiocfadh dóbhtha fosta lámh a chur ina mbás f...,"[31, 21, 22, 27, 16, 19, 14, 17, 21, 0, 17, 37...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"' na dhiaidh sin, bhí rud éigin do-chreidte ag...","[3, 0, 26, 14, 0, 17, 21, 22, 14, 22, 17, 21, ...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
filePath = "/content/drive/MyDrive/NLP/NLP_Assingnmnet05/data/train-05/train-target.txt"
train_target_data = load_data(filePath)

In [ ]:
Output_LTM = Language_Translation_Model()
Output_data = Output_LTM.preprocess_data(train_target_data[0:10000])
Output_data.head()

,sentence,sentence_array,sentence_array_OHE
0,"cinnte go leor, thiocfadh dóibh bás a fháil ar...","[15, 21, 25, 25, 30, 17, 0, 19, 26, 0, 23, 17,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,bhí sé follasach go raibh an poll sin ag foscl...,"[14, 20, 35, 0, 29, 34, 0, 18, 26, 23, 23, 13,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,d'fhéadfadh siad bás a fháil ar a bhruach agus...,"[16, 2, 18, 20, 34, 13, 16, 18, 13, 16, 20, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,thiocfadh dóibh fosta lámh a chur ina mbás féi...,"[30, 20, 21, 26, 15, 18, 13, 16, 20, 0, 16, 36...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,ina dhiaidh sin bhí rud éigin dochreidte agus ...,"[21, 25, 13, 0, 16, 20, 21, 13, 21, 16, 20, 0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
encoder_input_data = np.array([record for record in Input_data['sentence_array_OHE']])
decoder_input_data = np.array([np.concatenate((np.zeros((record.shape[0], 1), dtype='float32'), record[:, 1:]), axis=1) for record in Output_data['sentence_array_OHE']])
decoder_target_data = np.array([record for record in Output_data['sentence_array_OHE']])

### **Model Preparation**

In [ ]:
batch_size = 128
epochs=1
latent_dim=256
num_samples=10000

In [ ]:
encoder_inputs = Input(shape=(None, Input_LTM.length_all_Unique_Characters))
encoder = LSTM(latent_dim, return_state=True) # latent_dim = 256
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
#Decoder
decoder_inputs = Input(shape=(None, Output_LTM.length_all_Unique_Characters))

# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(Output_LTM.length_all_Unique_Characters, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Defining Model ------------------
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Training Model ------------------
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

1/1 [==============================] - 8s 8s/step - loss: 1.2344 - accuracy: 0.3334 - val_loss: 1.2413 - val_accuracy: 0.7151


**Predictions**

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_states_inputs = [Input(shape=(latent_dim,)), Input(shape=(latent_dim,))]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
test_input_seq = data.loc[0, 'sentence']
print(test_input_seq)
encoder_model.predict(test_input_seq)

**Save Model**

In [ ]:
model.save('s2s.h5')